In [ ]:
import pandas as pd
from estival.wrappers.nevergrad import optimize_model
from tb_incubator.plotting import plot_model_vs_actual
from tb_incubator.input import load_targets, load_param_info
from tb_incubator.calibrate import get_bcm
from tb_incubator.constants import ImplementCDR
from multiprocessing import cpu_count
pd.options.plotting.backend = "plotly"

In [ ]:
fixed_params = load_param_info()
all_targets = load_targets()
params = {
    'start_population_size': 1.0,
    "seed_time": 1805.0,
    "seed_num": 1.0,
    "seed_duration": 1.0
}
bcm = get_bcm(params, covid_effects=True, xpert_improvement=True, apply_cdr=ImplementCDR.ON_NOTIFICATION, apply_diagnostic_capacity=True)

In [ ]:
def calibrate_model_with_optimisation(bcm):
    """
    This function performs a model calibration using optimisation. 
    Calibration is performed using the estival package, which implements a wrapper for optimisation methods provided by the nevergrad package. 

    Args:
        bcm: the calibration model object (type BayesianCompartmentalModel) 
    """

    from nevergrad.optimization.differentialevolution import TwoPointsDE
    orunner = optimize_model(bcm, opt_class=TwoPointsDE, num_workers=cpu_count(), budget=4000)
    rec = orunner.minimize(4000)
    optimised_params = rec.value[1]    
   
    return optimised_params

In [ ]:
optimised_params = calibrate_model_with_optimisation(bcm)

In [ ]:
optimised_params

In [ ]:
# run the model with the optimised parameter set
res = bcm.run(optimised_params)
outs = res.derived_outputs

### Results

In [ ]:
fig = plot_model_vs_actual(
    outs, all_targets['notification'], "notification", "Notification", "Modelled vs Data", "Target data"
)
fig.update_xaxes(range=[2010, 2035])
#display_plot(fig, "genexpert_2points_notif")

In [ ]:
fig = plot_model_vs_actual(
    outs, all_targets['adults_prevalence_pulmonary_target'], "adults_prevalence_pulmonary", "Prevalence (Adult, Pulmonary)", "Modelled vs Data", "Target data"
)
fig.update_xaxes(range=[2010, 2035])